# Einleitung

Die Holznutzung in Deutschland ist nachhaltig. Seit 300 Jahren wächst mehr Holz nach, als verbraucht wird. Daher ist Scheitholz der wichtigste regenerative Energieträger zur Wärmegewinnung. Mit Holz kann CO2-neutral geheizt werden, weil während des Brennvorgangs nur so viel CO2 ausgestoßen wird, wie der Baum während seines Wachstums aus der Luft aufgenommen hat. Weitere Vorteile der Nutzung von Holz zur Wärmegewinnung sind kurze Transportwege, Unabhängigkeit und Versorgungssicherheit, sichere Lagerung und ungefährlicher Transport, Beschäftigung in der Region sowie ein geringer Energieaufwand bei Ernte und Aufbereitung.

Die HDG Bavaria, ein mittelständisches Unternehmen aus Bayern, produziert Holzheizungen seit den 1970er Jahren. Die HDG-Serie umfasst Heizkessel für jeden Bedarf und wird stetig erweitert. Der HDG F, der in dieser Ausarbeitung betrachtet wird, ist ein Holzvergaserkessel, der mit Scheitholz oder Spänebriketts betrieben werden kann. Der HDG F wird vor allem in Ein- und Mehrfamilienhäusern und landwirtschaftlichen Betrieben eingesetzt. Durch eine Bedienung, die ausschließlich von vorne erfolgt und eine große Füllöffnung bietet er dabei besonders viel Komfort.

<img src="pictures/Ofen von vorne.jpeg" height="400" alt="Ofen von vorne"/>

<img src="pictures/Ofen mit Holz.jpeg" height="400" alt="Ofen mit Holz"/>

Der in dieser Ausarbeitung thematisierte Kessel wird in einem Mehrfamilienhaus mit 4 Parteien verwendet. Ganzjährig sorgt er für die Warmwasseraufbereitung und wird bei Bedarf im Winter und in Übergangszeiten zum Heizen verwendet. Die Anlage kann nur betrieben werden, wenn zur richtigen Zeit das notwendige Holz verfügbar ist. Zu diesem Zweck gibt der Ofen selbst eine Prognose, wann er das nächste Mal Holz benötigt. Diese muss allerdings exakt sein, da ein zu spätes Nachlegen von Holz zu erheblichen Nachteilen führt. Erfahrungswerte zeigen, dass die ofeneigene Prognose nicht immer mit der Realität übereinstimmt. Deshalb soll in dieser Arbeit ein Modell entwickelt werden, welches einen möglich exakten spätesten Nachfüllzeitpunkt für den Ofen berechnet.

Daher wird diese Arbeit zunächst fachlich eingeordnet, indem der Holzofen und seine Funktionsweise beschrieben werden. Anschließend wird auf die Daten eingegangen, die der Ofen liefert und die wichtigsten Parameter werden erklärt. Danach wird mit der Arbeit an den Daten begonnen, indem die Daten geladen, integriert und vorverarbeitet werden. Anschließend folgt die Analyse, in der verschiedene Modelle zur Prognose mit dem ofeneigenen Wert verglichen und bewertet werden. Alle Schritte werden durch unterstützende Datenvisualisierungen und Erklärungen begleitet. Schließlich wird ein Gesamtüberblick über die gewonnenen Erkenntnisse und ein Ausblick gegeben.

[Heizen mit Holz](https://www.holzknecht-info.de/content/70/70/scheitholzkessel)

# Funktionsweise und Energiefluss

<img src="pictures/HDG F Schnittbild.png" width="400" alt="Schnittbild HDG F" />

Der Kessel des HDG F verfügt über eine große Füllöffnung, durch die der Füllschacht mit Holz befüllt wird. Im ersten Schritt wird das Holz vergast und anschließend bei hohen Temperaturen verbrannt. Währenddessen werden Primär- und Sekundärluftmengen durch Stellmotoren geregelt und Gase nachverbrannt bzw. abgesaugt. Die durch den Prozess entstandene Wärmeenergie wird mithilfe eines Wärmetauschers in einem Wasserpuffer gespeichert. Dieses Wasser wird direkt zum Heizen in die Heizkörper gepumpt. Um Warmwasser zu erzeugen, wird frisches Wasser über einen zweiten Wärmetauscher aufgeheizt. In der Aschelade sammeln sich die Verbrennungsrückstände, welche entsorgt werden müssen. Der Ofen wird durch ein Touch-Display bedient.

[Holzvergaser – maximale Wirkungsgrade mit zwei Brennkammern](https://www.thermondo.de/info/rat/erneuerbare-energie/holzvergaser/)

Im Diagramm ist der Energiefluss noch einmal schematisch abgebildet. Je nach Jahreszeit variieren die Anteile für Heizung und Warmwasser.

[Sankey Diagram in Python](https://plotly.com/python/sankey-diagram/)


In [81]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["Holz", "Luft", "Verlust", "Puffer", "Heizkörper", "Warmwasser"],
      color = ["saddlebrown", "lightblue", "grey", "green", "red", "blue"] 
    ),
    link = dict(
      source = [0, 1, 1, 3, 3, 3], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2, 3, 4, 5, 2],
      value = [100, 5, 95, 65, 15, 15],
      color = ["sandybrown", "lightgray", "azure", "tomato", "lightblue", "lightgrey"]
  ))])

fig.update_layout(title_text="Energieflussdiagramm Holzvergaser", font_size=15, title_font_color="white", paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

# Zustände

Das Aufheizen des Wassers im Puffer findet nicht kontinuierlich, sondern chargenweise statt. Deshalb befindet sich der Holzvergaser in verschiedenen Zuständen. Da diese im weiteren Verlauf der Arbeit relevant sind, werden sie hier kurz aufgeführt. In Abbildung 2 sind die Zustände und die möglichen Übergänge in einem Zustandsdiagramm dargestellt. Zu Beginn befindet sich der Holzvergaser im ausgeschalteten Zustand. Beim Befüllen des Ofens hat man zwei Möglichkeiten: Man kann den Ofen entweder direkt anzünden oder automatisch zünden lassen (Zündung bereit). Beim automatischen Zünden wartet der Holzvergaser, bis der Puffer den Ladegrad von 15 % unterschreitet und zündet dann das eingefüllte Holz mit einem Heißluftgebläse an. Ist die Zündung erfolgt, wechselt der Zustand in den Automatikmodus, bei dem das Holz vollständig verbrannt wird. Wenn nur noch Glut übrig ist, wechselt der Ofen in den Zustand Ausbrennen, indem man die 2 Möglichkeit hat, eine neue Charge Holz nachzulegen, sodass der Ofen im Automatikmodus weiter brennt. Andernfalls wechselt der Ofen in den ausgeschalteten Zustand.

Der Kessel kann durch automatische oder manuelle Zündung angeheizt werden.
Bei beiden Methoden startet die Nachlegetaste den Anheizvorgang. Zunächst müssen dann die Holzart, die Holzfeuchte, und die Zündart angegeben werden. Holzart und Holzfeuchte wirken sich auf die Brenndauer aus, daher sind diese zur Berechnung des nächsten Nachlegezeitpunktes entscheidend. Im nächsten Schritt wird der Nachlegebedarf angezeigt. Nun muss angegeben werden, wieviel Holz tatsächlich nachgelegt werden soll. Anhand dieser Angabe wird dann voraussichtlich nächste Nachlegezeitpunkt angezeigt. Nach diesen Angaben kann der Füllschacht geöffnet und befüllt werden.
Bei der automatischen Funktionsweise muss kleines Holz bzw. Papier oder Pappe vor das Zündloch gelegt und Holz darauf gestapelt werden. Die Tür wird geschlossen und die Zündung bestätigt. Der Kessel zündet dann automatisch, entweder sofort oder nach Aufforderung.

In [82]:
from graphviz import Graph

dot = Graph

# Datenverständnis

Die Arbeit baut im Wesentlichen auf zwei Datenquellen auf.
Der Holzofen zeichnet Daten auf und speichert diese auf einer SD-Karte. Alle 10 Sekunden werden insgesamt 62 Werte erfasst. Daten für den Holzofen, der hier betrachtet werden soll, liegen seit dem 12.11.2019 um 15:55 Uhr vor. Die letzten Daten wurden um ??? abgerufen. Nachfüllintervalle variieren zwischen 2-3 mal täglich im Winter und mehrmals wöchentlich im Sommer. Damit ergeben sich bereits über 1.000 aufgezeichnete Nachfüllungen, aus denen das Modell gespeist werden kann.

Als zweite Datenquelle werden externe Wetterdaten verwendet. In dieser Arbeit werden dazu die Wetterdaten von Meteostat über eine API abgerufen. Die Daten werden ursprünglich vom Deutschen Wetterdienst erhoben. Die API liefert stündliche Daten. Für das Modell werden die Daten in dem Zeitraum abgerufen, für den Ofendaten vorliegen. Die API liefert eine Vielzahl an Daten, für dieses Modell werden allerdings nur Temperatur, Windgeschwindigkeit und -richtung verwendet.

[Wetterrückblick und Klimadaten](https://meteostat.net/de/)